<a href="https://colab.research.google.com/github/laurence-lin/Kaggle_competition/blob/master/AnimeRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

%tensorflow_version 1.9
import tensorflow as tf
import keras
import lightgbm as lgb
import sklearn

import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc


`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.9`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
from google.colab import auth
auth.authenticate_user()

!gcloud config set project 'blind-detection'
!gsutil cp gs://anime_recommend/myanimelist.zip anime.zip
!unzip anime.zip

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Copying gs://anime_recommend/myanimelist.zip...
- [1 files][  1.9 GiB/  1.9 GiB]   41.5 MiB/s                                   
Operation completed over 1 objects/1.9 GiB.                                      
Archive:  anime.zip
  inflating: AnimeList.csv           
  inflating: UserAnimeList.csv       
  inflating: UserList.csv            
  inflating: anime_cleaned.csv       
  inflating: anime_filtered.csv      
  inflating: animelists_cleaned.csv  
  inflating: animelists_filtered.csv  
  inflating: users_cleaned.csv       
  inflating: users_filtered.csv      


In [0]:
'''
anime_list = pd.read_csv('AnimeList.csv') # list of animes, with features for each anime
user_animelist = pd.read_csv('UserAnimeList.csv') # list of records for each user watch a specific anime
user_list = pd.read_csv('UserList.csv')  # list of users, with features of the users behavior

# Filtered data: filter users 'with birthdate, and filled the gender & location'
anime_filter = pd.read_csv('anime_filtered.csv')
useranime_filter = pd.read_csv('animelists_filtered.csv')
userlist_filter = pd.read_csv('users_filtered.csv')
'''

# cleaned data: Truncate some below users, and adjust them with reasonable values:
# 1. Users that watch anime with large scale episode, but that anime don't have that much episode
# 2. Strange 'last visited date' for users, like: 1900
anime = pd.read_csv('anime_cleaned.csv')
ratings = pd.read_csv('animelists_cleaned.csv')
user = pd.read_csv('users_cleaned.csv')



In [39]:
#EDA

# Cleaned data

print(ratings.shape)
#print(ratings['my_start_date'].value_counts())
# Strange start date exists
#print(ratings['my_finish_date'].value_counts()[:10])
# Strange finished date: finished in 0000 years 

#print(ratings.columns)
#print(ratings['my_last_updated'].head())

# Convert my_last_updated to datetime type
#ratings['my_last_updated'] = pd.to_datetime(ratings['my_last_updated'], format = "%Y-%m-%d %H:%M:%S")
each_rate = ratings.groupby(['username', 'anime_id']).size()  # number of ratings each user gave to each anime
print(sorted(each_rate)[-7:]) # sort the 
# each user only rate each anime once


gc.collect()

(31284030, 11)
[1, 1, 1, 1, 1, 1, 1]


168

myscore: ratings of specific users for single anime.  
my_status: status of the watching for this anime 

1: watching  
2: complete  
3: on hold  
4: dropped  
5: plan to watch 
 
 Cleaned data: 31284030 records